In [ ]:
!pip install -qU llama-index datasets pinecone-client openai transformers


In [ ]:
import pandas as pd

df=pd.read_csv('your_dataset.csv')
df.head()

In [ ]:
df['category']=df['L1 - category']
df['subcategory']=df['L2 - sub-category']

df['gender']=df['L0 - gender']
df=df[['title','description','image_link','category','subcategory','product_type','brand','gender','price','season','color','condition']]




In [ ]:
df=df.dropna()


In [ ]:
df.info()


In [ ]:
import os
from llama_index import ServiceContext, LLMPredictor, OpenAIEmbedding, PromptHelper
from llama_index.llms import OpenAI
from llama_index.text_splitter import TokenTextSplitter
from llama_index.node_parser import SimpleNodeParser
from llama_index import VectorStoreIndex, SimpleDirectoryReader,KeywordTableIndex
from llama_index import set_global_service_context

In [ ]:
from pathlib import Path
from llama_index import download_loader
#Reading the data
PandasCSVReader = download_loader("PandasCSVReader")

loader = PandasCSVReader()
docs = loader.load_data(file=Path('./your_dataset.csv'))

In [ ]:
from llama_index import Document
from llama_index.node_parser import SentenceSplitter
import tiktoken
#Creating nodes from the documents
node_parser=SimpleNodeParser()

nodes = node_parser.get_nodes_from_documents(
    docs
)
nodes[0]

In [ ]:
#Specify the models are going to use in the service context to create our index
llm = OpenAI(model='gpt-3.5-turbo', temperature=0, max_tokens=256)

embed_model = OpenAIEmbedding()

prompt_helper = PromptHelper(

  context_window=4096,

  num_output=256,

  chunk_overlap_ratio=0.2,

  chunk_size_limit=None

)

service_context = ServiceContext.from_defaults(

  llm=llm,

  embed_model=embed_model,

  node_parser=node_parser,

  prompt_helper=prompt_helper

)

In [ ]:
#creating the index
index = VectorStoreIndex.from_documents(
    docs,
    service_context = service_context
    )

In [ ]:
# Persisting to disk
index.storage_context.persist(persist_dir="Directory_to_Save_your_index")